# A-B Testing

Firstly I want to add some useful resources to it:
- https://blog.duolingo.com/improving-duolingo-one-experiment-at-a-time/ 
- https://www.kaggle.com/code/ekrembayar/a-b-testing-step-by-step-hypothesis-testing

The first link Explain why A/B test are useful for Duolingo, the second link is more of what I am going to do here. 
I also added the second link because probably I will use some of the code use there to do the analysis and I think it is  respecful to put the link of the actual source :) 

So first of all A/B test it is just a way to test if two group has a difference in a target or targets value. Generally, there is a control group and a group that experience some "new treatment" that we want to test.
There is a very common test that is the **T-test** that you may be wondering why we just don't use that test and we test exactly that. The problem with that is that that particular test has as all the test usually has **some assumption that need to be fulfill** Those assumption are:
1) The groups that are we comparing the target comes from a normal distribution
2) The groups have the same variance

Pretty big assumption if you ask me!, especially the last one.

So if we want to use this test we need to check both of these requierement. So we need to:
1) test normality with a test, we will use Shapiro test.
2) Use Leve Test for test homogeneity of variance (equal variance)

If and only if we have a check in both assumption we use the T-test.
If (1) is true but (2) is not true (using Levene test) We use Welch Test.
If (1) is not true we directly jump to use Mann Whitney U Test.

There is some veeery interesting debate on this test that I highly recommend you to read more in case you want to know more about this tests (I would recommend to read them in the order presented!):

- https://stats.stackexchange.com/questions/2492/is-normality-testing-essentially-useless
- https://stats.stackexchange.com/questions/121852/how-to-choose-between-t-test-or-non-parametric-test-e-g-wilcoxon-in-small-sampl
- https://stats.stackexchange.com/questions/151663/normality-and-when-to-use-t-test-vs-mann-whitney-u-test

Some interesting concepts that you may encounter there and are interesting to ponder about:
+ Power vs Robustness
+ skew vs kurtotic and how impact inference about mean vs variance
+ What happened to this assumption for n small, large or very large. Other important topics on the same line: CLT (central limit theorem) and small variation on the normal distribution and Saphiro test.


We will use the same data that use the second link: https://www.kaggle.com/code/ekrembayar/a-b-testing-step-by-step-hypothesis-testing/input